In [2]:
# ===================== INSTALL DEPENDENCIES =====================
!pip install -q langchain sentence-transformers faiss-cpu pypdf groq langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

Imagine you're assembling a LEGO robot. Each block (like arms, wheels, sensors) has a job. You connect blocks together, and the robot works — step by step.

####In LangChain v0.3, these blocks are called Runnables:
1. One block splits documents
2. One block embeds text
3. One block talks to the AI model
4. One block retrieves docs
5. One block formats the prompt

####You can chain them together like:
load_pdf ➡️ split_text ➡️ embed ➡️ retrieve ➡️ prompt ➡️ call LLM

####That's the power of Runnable. Let's Break Down Your RAG Pipeline Using Runnables
I'll now rewrite your pipeline using LangChain v0.3’s Runnable architecture and explain only the Runnable related parts.

In [3]:
# ================== IMPORTS ==================
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda, RunnablePassthrough
from langchain_groq import ChatGroq

In [4]:
# ================== LOAD & SPLIT PDF ==================
loader = PyPDFLoader("/content/solid-python.pdf")
documents = loader.load_and_split()

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)


In [5]:
# ================== EMBEDDINGS + VECTORSTORE ==================
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding_model)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

/tmp/ipython-input-5-204823150.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mode

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# ================== DEFINE LLM ==================
from google.colab import userdata
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=userdata.get('GROQ_API_KEY')
)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x784eb2750210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x784eb27c6e90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
# ================== DEFINE PROMPT ==================
prompt = PromptTemplate.from_template(
    "Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}"
)

In [8]:
# ================== RAG CHAIN USING RUNNABLE ==================

# Step 1: Get relevant docs
retriever_runnable = RunnableLambda(lambda x: retriever.get_relevant_documents(x["question"]))

# Step 2: Format prompt
prompt_runnable = prompt | RunnableLambda(lambda x: {"messages": [x]})

# Step 3: Final RAG chain
rag_chain = (
    RunnableMap({
        "context": retriever_runnable,
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
)

In [11]:
# ================== RUN THE CHAIN ==================
question = "What is the main objective of the document?"
response = rag_chain.invoke({"question": question})

In [12]:
print(response.content)

The main objective of the document is to discuss and explain the SOLID principles of software design, with the goal of maintaining software quality over time. The document appears to be a presentation or lecture on the topic, covering various aspects of the SOLID principles, including the Single Responsibility Principle, Liskov Substitution Principle, and Dependency Inversion Principle.


| **Component**         | **LangChain Runnable**          | **Analogy**                                      | **Purpose**                                                                |                                    |                                                            |
| --------------------- | ------------------------------- | ------------------------------------------------ | -------------------------------------------------------------------------- | ---------------------------------- | ---------------------------------------------------------- |
| `RunnableLambda`      | `RunnableLambda(lambda x: ...)` | A **custom logic LEGO block** you snap in        | Wraps any Python function to turn it into a Runnable                       |                                    |                                                            |
| `RunnableMap`         | `RunnableMap({...})`            | A **multi-socket splitter** 🧩                   | Takes 1 input and creates multiple outputs like "question", "context"      |                                    |                                                            |
| `RunnablePassthrough` | `RunnablePassthrough()`         | A **transparent pipe**                           | Passes input directly as output without changing it                        |                                    |                                                            |
| \`                    | \` (pipe) operator              | \`A                                              | B\`                                                                        | **Plug A's output into B's input** | Used to connect blocks into a chain (like a conveyor belt) |
| `invoke()`            | `.invoke(input)`                | Like **pressing the START button** on your robot | Sends your input through the whole pipeline and gives you the final output |                                    |                                                            |
